In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.controller.combined_robot import *
from pkg.project_config import *

In [2]:
CONNECT_INDY = False

MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

In [3]:
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP),
    RobotConfig(1, RobotType.indy7, ((0.172,0,0.439), (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})]
              , connection_list=[False, CONNECT_INDY])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# xyz_rpy_robots = {"kmb0": ((0,0,0), (0,0,0)), "indy1": ((0,0,0), (0,0,np.pi))}
# crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
gscene.show_pose(-crob.home_pose)

In [5]:
# gscene.show_pose(-crob.home_pose + ([0.2,0.2,np.pi/3,]+[0]*6))

## set workspace

In [6]:
gscene.set_workspace_boundary( -5, 5, -5, 5, -0.1, 1.75)

Please create a subscriber to the marker


## define robot and planning scene

In [7]:
from pkg.planning.scene import *
from demo_utils.environment import *

CHAIR_NAME = "chair0"
CHAIR_DIM = (0.4,0.4,0.5)

MOBILE_NAME = "kmb0"
HOLD_LINK = "kmb0_platform"
HOLD_XYZ = (0.445 + 0.35, 0, CHAIR_DIM[2]/2)
HOLD_RPY = (0, -np.pi/2, -np.pi/2)
HOLD_NAME = "hold0"

ROBOT_NAME = "indy1"
TOOL_LINK = "indy1_tcp"
TOOL_NAME = "brush_face"

HOME_POSE = -crob.home_pose

gtems_robot = s_builder.add_robot_geometries(color=(0, 1, 0, 0.5), display=True, collision=True)
viewpoint = add_cam(gscene, tool_link=TOOL_LINK)
add_indy_tool_kiro(gscene, tool_link=TOOL_LINK, face_name=TOOL_NAME, zoff=-0.04)

pscene = PlanningScene(gscene, combined_robot=crob)

## planning pipeline

In [8]:

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)


from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [9]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.motion_filters = [GraspChecker(pscene)]

   Use a production WSGI server instead.
 * Debug mode: off


## Add action points

In [10]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [11]:
CLEARANCE = 0.001
gscene.create_safe(gtype=GEOTYPE.BOX, name="floor", link_name="base_link",
                   dims=(10,10,0.1), center=(0,0,-0.05-CLEARANCE), 
                   rpy=(0,0,0), color=(0.9, 0.9, 0.9, 0.9), display=True,
                   collision=True, fixed=True)
floor = pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)

brush_face = pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(-gscene.NAME_DICT['brush_face'].dims[0]/2,0,0), 
                     rpy=(0,np.pi/2*1,0))

gscene.create_safe(gtype=GEOTYPE.SPHERE, name=HOLD_NAME, link_name=HOLD_LINK,
                   dims=(0.01,) * 3, center=HOLD_XYZ, rpy=HOLD_RPY, color=(1, 0, 0, 0.5), display=True,
                   collision=False,
                   fixed=True)
holder = pscene.create_binder(bname=HOLD_NAME, gname=HOLD_NAME, _type=Gripper2Tool, point=(0, 0, 0), rpy=(0, 0, 0))
holder.redundancy = {}

## Add chair

In [12]:
chair_loc = (1, -3.5, CHAIR_DIM[2]/2)

obs_all = 10

obj_all = 3

for i_c in range(obs_all):
    CHAIR_NAME_CUR = CHAIR_NAME[:-1]+"%01d"%i_c
    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name=CHAIR_NAME_CUR, link_name="base_link",
                       dims=CHAIR_DIM, center=np.add(chair_loc, [0,0.75*i_c, 0]), rpy=(0,0,0),
                       color=(1,0,0,1) if i_c==0 else ((1,1,0,1) if i_c<obj_all else (0.3, 0.3, 0.3, 0.3)), 
                       display=True, collision=True, fixed=False)
    if i_c < obj_all:
        chair = pscene.create_subject(oname=CHAIR_NAME_CUR, gname=CHAIR_NAME_CUR, _type=CylinderObject, 
                                      GRASP_WIDTH_MIN=CHAIR_DIM[0]-0.1, GRASP_WIDTH_MAX=CHAIR_DIM[0]+0.1, 
                                      GRASP_DEPTH_MIN=CHAIR_DIM[0]/2, GRASP_DEPTH_MAX=CHAIR_DIM[0]/2)
#         chair.action_points_dict[CHAIR_NAME_CUR+"_side_g"].redundancy = {'u': (-np.pi, np.pi)}
        chair.action_points_dict[CHAIR_NAME_CUR+"_side_g"].redundancy = {'u': (-0, 0)}

## [Test] pick and place chair

In [13]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="goal", link_name="base_link",
                   dims=(1.0, 1.0, 0.1), center=(3,0,-0.05-CLEARANCE), rpy=(0,0,0),
                   color=(0.8, 0.3, 0.3, 0.3), display=True,
                   collision=True, fixed=True)

goal = pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=None)

In [14]:
initial_state = pscene.initialize_state(HOME_POSE)
goal_nodes = [('goal',)+rest for rest in list(product(["goal", "floor"], repeat=obj_all-1))]
goal_nodes = [('goal',)+(("floor",)*(obj_all-1))]

In [15]:
# from pkg.planning.task.rrt import TaskRRT

# tplan = TaskRRT(pscene)
# ppline.set_motion_planner(mplan)
# ppline.set_task_planner(tplan)

# ppline.search(initial_state, goal_nodes, verbose=True,
#               timeout=1, timeout_loop=100, multiprocess=False,
#               add_homing=True)
# schedules = ppline.tplan.find_schedules(False)
# schedules_sorted = ppline.tplan.sort_schedule(schedules)
# snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
# ppline.play_schedule(snode_schedule)


In [16]:
from pkg.planning.pddlstream.plan_rnb import solve_in_pddlstream

goal_pairs=[(CHAIR_NAME, 'goal'), (CHAIR_NAME[:-1]+"1", "floor"), (CHAIR_NAME[:-1]+"2", "floor")]
TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS = 1, 100, 100, 30
GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO = 100, 100, True, 100
VERBOSE=True


res, plan, log_dict = solve_in_pddlstream(pscene, mplan, MOBILE_NAME, HOLD_NAME, HOME_POSE, goal_pairs,
                        TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS,
                        GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO,
                        use_pybullet_gui=False, USE_MOVEIT_IK=True, VERBOSE=VERBOSE)

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
Objects: {1L: 'brush_face_col', 2L: 'hindge1_col', 3L: 'indy1_link1_Cylinder_0', 4L: 'hindge0_col', 5L: 'chair5', 6L: 'leftwall_ws', 7L: 'chair4', 8L: 'chair7', 9L: 'chair9', 10L: 'chair8', 11L: 'goal', 12L: 'floor', 13L: 'indy1_link2_Cylinder_0', 14L: 'indy1_link2_Cylinder_1', 15L: 'chair1', 16L: 'chair0', 17L: 'ceiling_ws', 18L: 'adapter_col', 19L: 'floor_ws', 20L: 'frontwall_ws', 21L: 'rightwall_ws', 22L: 'cam_col', 23L: 'backhead_col', 24L: 'chair6', 25L: 'chair2', 26L: 'indy1_link4_Cylinder_0', 27L: 'body_col', 28L: 'hinge_bar_col', 29L: 'backwall_ws', 30L: 'indy1_link6_Cylinder_0', 31L: 'indy1_link5_Cylinder_0', 32L: 'chair3', 33L: 'indy1_link0_Box_0', 34L: 'brushbase_col', 35L: 'indy1_link3_Cylinder_0'}
================ MAKE PROBLEM ======================
IK checkers: ['GraspChecker']
MP checkers: []
timeout motion : 1
Robot: 0
Movable: [25L, 15L, 16L]
Fixed: [1, 2, 3, 4, 5, 6, 7, 8,

iter=0, outs=1) sample-grasp:(16)->[(g0)]
iter=inf, outs=1) inverse-kinematics:(16, p2, g0)->[(q1, c0)]
iter=inf, outs=1) test-cfree-traj-pose:(c0, 15, p1)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c0, 25, p0)->[()]
iter=0, outs=1) sample-pose:(16, 11)->[(p3)]
iter=inf, outs=1) inverse-kinematics:(16, p3, g0)->[(q2, c1)]
iter=inf, outs=1) test-cfree-traj-pose:(c1, 25, p0)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c1, 15, p1)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c0, 16, p2)->[()]
iter=inf, outs=1) test-cfree-pose-pose:(16, p3, 15, p1)->[()]
iter=inf, outs=1) test-cfree-pose-pose:(16, p3, 25, p0)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(16, p2, g0, 15, p1)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(16, p2, g0, 25, p0)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(16, p3, g0, 15, p1)->[()]
iter=inf, outs=1) test-cfree-approach-pose:(16, p3, g0, 25, p0)->[()]
sucject/actor: chair0 / hold0
check / plan: True / True
iter=inf, outs=1) plan-free-motion:(q0,

In [20]:
from pkg.planning.pddlstream.plan_rnb import *
play_pddl_plan(pscene, gripper=pscene.actor_dict["hold0"], initial_state=initial_state, 
               body_names=log_dict['body_names'], plan = plan)